# **0. 패키지 설치**

In [ ]:
!pip install openai
!pip install langchain
!pip install -U langchain-community
!pip install pypdf
!pip install faiss-cpu
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 76.0 MB/s eta 0:00:00


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# **0. 구글 드라이브 마운트 & api key**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import glob
import os

Mounted at /content/drive


In [ ]:
import os
import openai
from langchain.chat_models import ChatOpenAI

#  API 키 설정
OPENAI_API_KEY = ""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
client = openai.OpenAI(api_key=OPENAI_API_KEY)
openai.api_key = OPENAI_API_KEY

# **1. 문서 업로드 & Retriever**

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# 1. 📥 PDF 로딩 (하위 폴더 포함) + 출처 추적
def load_all_pdfs(root_dir):
    all_docs = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".pdf"):
                full_path = os.path.join(dirpath, filename)
                loader = UnstructuredPDFLoader(full_path)
                docs = loader.load()

                for doc in docs:
                    # 📎 source: 상대 경로 기준으로 저장 (예: RAG/작품A.pdf)
                    relative_path = os.path.relpath(full_path, root_dir)
                    doc.metadata["source"] = relative_path
                    doc.page_content = doc.page_content.strip().replace("\n", " ").replace("  ", " ")

                all_docs.extend(docs)
    return all_docs

# 2. 텍스트 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

# 3. 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 4. 배치 임베딩
def embed_in_batches(docs, batch_size=100):
    vectorstore = None
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        print(f"🔄 Embedding batch {i} ~ {i + len(batch)} / {len(docs)}")
        if vectorstore is None:
            vectorstore = FAISS.from_documents(batch, embedding=embeddings)
        else:
            vectorstore.add_documents(batch)
    return vectorstore

# 5. 저장 경로
VECTORSTORE_PATH = "/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/문학/faiss_index_답변해설"

# 6. 벡터스토어 생성 또는 로딩
def build_vectorstore():
    root_dir = "/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/문학/data/RAG"
    all_docs = load_all_pdfs(root_dir)
    split_docs = text_splitter.split_documents(all_docs)
    vectorstore = embed_in_batches(split_docs)
    vectorstore.save_local(VECTORSTORE_PATH)
    return vectorstore

# 7. 리트리버 생성
def get_retriever():
    if os.path.exists(VECTORSTORE_PATH):
        print("📁 기존 벡터스토어 불러오는 중...")
        vectorstore = FAISS.load_local(VECTORSTORE_PATH, embeddings, allow_dangerous_deserialization=True)
    else:
        print("🧠 새 벡터스토어 생성 중...")
        vectorstore = build_vectorstore()

    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 5,
            "score_threshold": 0.5
        }
    )
    return retriever

# 🔄 최종 retriever
retriever = get_retriever()

/tmp/ipython-input-5-2237303118.py:37: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


📁 기존 벡터스토어 불러오는 중...


# **2. 개념 답변 QA chain**

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 개념 설명 체인
concept_prompt = ChatPromptTemplate.from_template("""
너는 한국의 대학수학능력시험의 국어 과목 중 문학의 개념에 대해 설명해주는 튜터야. 아래 질문에 대해 간단명료하게 설명해줘.

질문:
{question}
""")

concept_chain = concept_prompt | llm | StrOutputParser()

/tmp/ipython-input-6-364772018.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)


# **3. 문제 답변&해설 QA 체인**
- RAG를 활용해서 Retriever에서 찾은 문서 내용들을 prompt로 전달해서 참고하라고 llm에게 전달.
- prompt <- 지문, 참고자료, 문제

In [ ]:
# 문제 기반 QA RAG 체인
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnableLambda

# LLM 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

# 문제 기반 QA 체인용 프롬프트
qa_prompt = ChatPromptTemplate.from_template("""
당신은 한국 수능 국어 문학 전문 튜터입니다.

다음은 수능 국어 문학 객관식 문제입니다. ①, ②, ③, ④, ⑤ 중 하나를 고르는 객관식 문제이며,
정답은 반드시 지문 및 <보기>의 정확한 분석에 근거해 판단해야 합니다.

🟨 반드시 지켜야 할 분석 기준:

1. 지문 분석을 중심으로 판단하며, **선택지의 내용이 지문과 일치하는지** 엄밀히 검토하세요.
2. 문제에 <보기>가 있다면, <보기> 속 설명(구조, 시점, 표현, 인물 해석 등)을 **지문에 어떻게 적용했는지** 구체적으로 분석하세요.
3. **서술 방식, 인물 심리 표현, 문체, 시점 변화, 지시 표현, 병렬 구조** 등은 서술상·표현상 특징 문제에서 핵심 근거입니다.
4. 참고 자료는 반드시 보조적 용도로만 활용합니다. 절대 지문을 덮어쓰거나 대체해서는 안 됩니다.

📌 답변 형식:

[정답]
- (①, ②, ③, ④, ⑤ 중 하나)

[해설]
- 문제에서 요구한 핵심 요소(예: 표현 방식, 구조, 시점 등)에 따라 왜 정답인지 논리적으로 설명합니다.
- 지문 및 <보기>의 문장을 **직접 인용**하여 명확한 판단 근거를 제시하세요.
- 나머지 오답 선택지들은 각각 왜 틀렸는지를 간단히 설명하세요.

📖 [지문]
{context}

📚 [참고 자료] — 필요 시만 사용 (retriever 제공):
{reference}

🙋‍♂️ [문제 및 <보기>]
{question}
""")

def format_with_retrieved_docs(inputs):
    question = inputs["question"]
    context = inputs["context"]

    # 🔍 관련 문서 검색 (context와 question 모두 기준으로 검색)
    retrieved_docs = retriever.get_relevant_documents(f"{context}\n\n{question}")
    retrieved_context = "\n\n".join(doc.page_content for doc in retrieved_docs)

    return {
        "context": context,           # 지문 및 <보기>
        "reference": retrieved_context,  # 보조 자료
        "question": question
    }

# RAG QA 체인: 문항 지문 중심 + retriever 참고
rag_qa_chain = (
    RunnableLambda(format_with_retrieved_docs)
    | qa_prompt
    | llm
    | StrOutputParser()
)

# **4. 문제 vs 개념 분류 함수**

In [ ]:
def is_problem_question(question: str) -> bool:
    classification_prompt = ChatPromptTemplate.from_template("""
다음 질문이 문학 개념 질문인지, 지문 기반 문제인지 판별해줘. '개념' 또는 '문제' 중 하나로만 답해.

질문:
{question}
""")
    chain = classification_prompt | llm | StrOutputParser()
    result = chain.invoke({"question": question})
    return "문제" in result.strip()

# **5. 최종 질문 처리 함수**
문제 질문인지 개념 질문인지 분류 -> 개념 답변 / 문제 답변 해설

In [ ]:
def tutor_response(question: str, passage: str = None):
    """
    문학 질문 또는 문항에 대한 응답을 처리합니다.

    Parameters:
    - question: 수험생 질문 또는 문학 개념 질문
    - passage: 선택적 입력. 문항 지문이 있는 경우 함께 제공
    """
    if is_problem_question(question):
        if not passage:
            print("❗ 오류: 문학 문제 풀이에는 지문(passage)이 필요합니다.")
            return

        print("📘 [문제에 대한 정답 및 해설]")

        # rag_qa_chain에 지문과 문제를 분리해서 입력
        response = rag_qa_chain.invoke({
            "context": passage,
            "question": question
        })
        print(response)

    else:
        print("📘 [문학 개념 설명]")
        response = concept_chain.invoke({"question": question})
        print(response)

# **6. 이미지(지문/문항)에서 텍스트 추출**
- 지문 이미지에서 텍스트 추출할때 비용 문제 발생

### **6.1 지문 텍스트 추출**
이미지 5등분 -> gpt-4o로 텍스트 추출 -> gpt-4o가 추출한 텍스트와 이미지를 비교하면 특수문자 삽입 -> gpt-4o가 특수문자 검토 -> gpt-4o가 지문 범위([A], [B] 등) 삽입

In [ ]:
from PIL import Image
import openai
import base64
import io
from typing import List, Tuple

# ✅ 이미지 → base64
def image_to_base64(image: Image.Image) -> str:
    buffer = io.BytesIO()
    image.save(buffer, format="PNG")
    return f"data:image/png;base64,{base64.b64encode(buffer.getvalue()).decode()}"

# ✅ 이미지 수직 분할
def split_image_vertically(image: Image.Image, parts: int = 5) -> List[Image.Image]:
    width, height = image.size
    part_height = height // parts
    return [
        image.crop((0, i * part_height, width, height if i == parts - 1 else (i + 1) * part_height))
        for i in range(parts)
    ]

# ✅ GPT에게 OCR 시키는 함수
def gpt_ocr_text(image: Image.Image) -> str:
    base64_img = image_to_base64(image)

    system_prompt = """
    너는 수능 국어 문학의 지문 OCR 텍스트 추출 전문가야.

    아래 이미지를 보고 **OCR 텍스트를 그대로 추출**해.
    텍스트 추출만 하고, 절대 가공하거나 설명하지 마.

    📌 반드시 지킬 것:
    - 줄바꿈은 이미지에 보이는 그대로 살려야 해.
    - 띄어쓰기, 특수문자, 괄호, 마침표 등 모든 문장 부호도 그대로 유지해야 해.
    - 해석이나 부연 설명 없이 **순수한 OCR 결과만 출력**해야 해.
    """

    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt.strip()},
                {
                    "role": "user",
                    "content": [
                        {"type": "image_url", "image_url": {
                            "url": base64_img,
                            "detail": "high"
                        }}
                    ]
                }
            ],
            temperature=0,
            max_tokens=16000
        )
        output = response.choices[0].message.content.strip()
        if not output or "죄송하지만" in output:
            raise ValueError("GPT OCR 실패 또는 결과 없음")
        return output
    except Exception as e:
        return f"[❌ GPT OCR 실패]: {str(e)}"

# ✅ GPT-4o로 특수기호&괄호 삽입
def refine_text_with_gpt(image: Image.Image, ocr_output: str) -> str:
    base64_img = image_to_base64(image)

    system_prompt = """
    너는 수능 국어 문학 지문 정리 전문가야.

    다음은 OCR로 추출한 지문과 원본 이미지야. 너는 이 두 정보를 비교해서 다음과 같이 OCR 결과를 정확하게 수정해야 해.

    📌 반드시 지켜야 할 규칙:

    1. ✅ 이미지 안에서 특수기호(원형 문자/알파벳)를 정확히 식별하고, OCR 텍스트의 알맞은 위치에 삽입해야 해.
      - OCR 결과만 보면 안 돼. 반드시 이미지에서 특수기호 위치를 기준으로 판단해야 해.

    2. ✅ 삽입해야 할 특수기호는 다음과 같아:
      - 한글 원형 문자: ㉠, ㉡, ㉢, ㉣, ㉤
      - 원형 알파벳: ⓐ, ⓑ, ⓒ, ⓓ, ⓔ

    3. ✅ **모든 특수기호 뒤에는 밑줄로 강조된 문장이 반드시 있고**, 그 **강조된 문장 전체를 괄호 '( )'로 반드시 감싸야 해.**
      - 특수기호와 괄호는 붙여서 작성해: 예) ⓐ(강조된 문장)
      - 괄호는 해당 문장의 시작과 끝을 정확히 감싸야 하며, 줄바꿈이나 공백이 있어도 전체를 포함해야 해.

    4. ✅ 한 특수기호에 해당하는 강조 문장이 **여러 줄에 걸쳐 있더라도** 괄호로 정확히 감싸야 해.
      - 중간 줄바꿈이나 공백이 있더라도 강조 문장 전체가 괄호 안에 들어가야 해.

    5. ❌ 특수기호가 없는 문장은 수정하지 마.
      ❌ 철자 오류, 띄어쓰기 오류 등 OCR 자체 오류도 고치지 마.

    6. ✅ 출력은 반드시 지문 전체를 포함해야 하며, 특수기호 + 강조문장 부분만 수정해야 해.
      ❌ 설명, 해설, 추가 정보 없이 **지문 전체만 출력**해야 해.


     ⚠️ 반드시 특수기호 뒤에는 밑줄로 강조된 부분이 괄호'()'로 감싸져 있어야해.
     ⚠️ 특수기호 뒤에 괄호가 없다면 다시 밑줄로 강조된 부분을 다시 찾고 괄호를 추가해줘.
    """

    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt.strip()},
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": f"OCR 추출 결과:\n{ocr_output.strip()}"},
                        {"type": "image_url", "image_url": {
                            "url": base64_img,
                            "detail": "auto"
                        }}
                    ]
                }
            ],
            temperature=0.1,
            max_tokens=16000  # 가능한 최대값 사용
        )
        output = response.choices[0].message.content.strip()
        if not output or "죄송하지만" in output or "도와드릴 수 없습니다" in output:
            raise ValueError("GPT 응답 오류 또는 내용 없음")
        return output
    except Exception as e:
        return f"[❌ GPT 정교화 실패]: {str(e)}"

In [ ]:
# ✅ 특수기호 위치 점검 및 보정 요청
def verify_special_symbols(original_image: Image.Image, restored_text: str) -> str:
    base64_img = image_to_base64(original_image)

    system_prompt = """
      너는 수능 국어 문학의 지문 OCR 복원 전문가야.

      다음은 OCR로 추출한 지문과 원본 이미지야. 너는 이 두 정보를 비교해서 다음과 같이 OCR 결과를 정확하게 수정해야 해.
      추출한 지문의 특수기호(㉠, ㉡, ㉢, ㉣, ㉤, ⓐ, ⓑ, ⓒ, ⓓ, ⓔ 등)가 원본 이미지의 특수기호와 다를 수 있어서 수정해줘.

      원본 이미지를 보고
      - 문장 앞 기호가 이미지와 정확히 일치하는지 확인하고,
      - 잘못된 기호는 올바른 특수기호로 수정해.
      - 이미지에는 특수기호가 없는데 텍스트에는 있는 경우는 특수기호를 삭제해줘.
      - **추출한 지문의 특수기호 뒤에 괄호'('가 없다면 이미지에서 밑줄로 강조된 부분을 찾아서 괄호로 감싸서 지문을 수정해줘.**

      ⚠️ ①, ②, ③, ④, ⑤ 이런 원형 숫자 특수기호는 지문에 있을 수 없어 반드시 다시 올바른 특수기호로 수정해줘.
      ⚠️ 동일한 특수기호가 또 나올 수 없어 다시 올바른 특수기호로 수정해줘.
      ⚠️ 반드시 특수기호 뒤에는 밑줄로 강조된 부분이 괄호'()'로 반드시 감싸져 있어야해.
      ⚠️ 추출한 지문의 특수기호 뒤에 괄호가 없다면 원본 이미지에서 특수기호 뒤에 다시 밑줄로 강조된 부분을 다시 찾고 지문 텍스트에 괄호'()'를 추가해줘.
      ⚠️ 이미지 기호 위치를 꼭 확인해 판단하고,
      ⚠️ 출력은 반드시 지문 전체를 출력해.
    """

    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt.strip()},
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": f"복원된 지문 일부:\n{restored_text.strip()}"},
                        {"type": "image_url", "image_url": {
                            "url": base64_img,
                            "detail": "high"
                        }}
                    ]
                }
            ],
            temperature=0.1,
            max_tokens=16000
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"[❌ GPT 기호 검토 실패]: {str(e)}"

In [ ]:
# 이미지 우측 지문 범위 표시
def insert_passage_brackets_with_gpt(image: Image.Image, ocr_text: str) -> str:
    base64_img = image_to_base64(image)

    system_prompt = """
    너는 수능 국어 문학의 지문 OCR 복원 전문가야.

    다음은 OCR로 추출한 지문과 원본 이미지야.
    원본 이미지의 **좌측 혹은 우측에 [A], [B], [C] 등의 표시와 함께 특정 지문 범위를 나타내는 선들이 있는 경우가 있어.**
    이 시각적 정보는 어떤 지문 구간이 문제 풀이에서 중요한지를 알려주는 단서야.

    🔍 너의 임무는 다음과 같아:

    1. 지문 이미지의 맨 왼쪽과 맨 오른쪽을 잘 살펴보고,
      특정 지문 범위를 나타내는 선과 함께 [A], [B], [C] 등이 지문 내 **어디서부터 어디까지를 가리키는지** 판단해.
        - 각 범위는 반드시 **1줄 이상**이 되도록 하고, **문장이 끊기지 않게 자연스럽게 포함**해야 해.
        - 문장의 시작이나 끝이 잘리지 않도록, 지문 흐름에 맞게 해당 **시작 줄과 끝 줄 전체를 포함**해야 해.

    2. 만약 특정 지문 범위를 나타내는 선이 있다면, 판단된 범위를 지문 중 그대로 유지하되 아래 예시처럼 `[A]{}`로 감싸 강조해줘:
        ...
        [A] {
            (해당 범위 시작 줄)
            ...
            (해당 범위 끝 줄)
        }
        ...

    ⚠️ 유의사항:
    - 줄 단위로 판단하되, 의미 단위(문장 구조)를 최대한 유지해야 해.
    - 특정 지문 범위의 시작과 끝 위치는 이미지 오른쪽의 선과 시각적으로 정렬된 지문 줄을 찾아 판단해야 해.
    - 줄바꿈, 공백, 괄호, 특수기호 등은 OCR 텍스트 스타일을 그대로 유지해.
    """

    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt.strip()},
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": f"OCR 결과:\n{ocr_text.strip()}"},
                        {"type": "image_url", "image_url": {
                            "url": base64_img,
                            "detail": "high"
                        }}
                    ]
                }
            ],
            temperature=0.1,
            max_tokens=16000
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"[❌ GPT 지문 범위 삽입 실패]: {str(e)}"

In [ ]:
import os
from PIL import Image

def run_split_pipeline(image_path: str, parts: int = 4):
    print("📂 이미지 로딩 중...")
    image = Image.open(image_path)

    # 1. 이미지 수직 분할
    print(f"🔀 이미지 {parts}등분 중...")
    split_images = split_image_vertically(image, parts=parts)

    # 2. 각 분할 이미지 OCR
    ocr_parts = []
    for idx, part_img in enumerate(split_images):
        print(f"🔍 [{idx + 1}/{parts}] 분할 이미지 OCR 중...")
        part_text = gpt_ocr_text(part_img)
        ocr_parts.append(part_text)

    # 3. OCR 결과 병합
    combined_ocr = "\n".join(ocr_parts).strip()

    # 4. GPT로 정교화(특수기호&괄호 삽입)
    print("🔧 GPT 정교화 단계 진행 중...")
    refined_text = refine_text_with_gpt(image, combined_ocr)

    # 5. 특수기호 위치 검토 및 보정 => 성공
    print("🧠 특수기호 위치 검토 및 보정 중...")
    symbol_corrected_text = verify_special_symbols(image, refined_text)

    # 6. 이미지 우측 지문 범위 표시
    print("🗂️ GPT 지문 범위 표시([A], [B] 등) 삽입 중...")
    final_result = insert_passage_brackets_with_gpt(image, symbol_corrected_text)

    return final_result.strip()

### **6.2 문제 텍스트 추출**

In [ ]:
def extract_question(question_path):
    import mimetypes

    # 파일 확장자에 맞게 MIME 타입 추정
    mime_type, _ = mimetypes.guess_type(question_path)
    if not mime_type:
        mime_type = "image/png"  # 기본값 fallback

    with open(question_path, "rb") as f:
        base64_img = base64.b64encode(f.read()).decode("utf-8")

    image_url = f"data:{mime_type};base64,{base64_img}"

    system_prompt = """
    너는 수능 국어 문학 문제 이미지를 텍스트로 정확하게 복원하는 OCR 모델이야.
    입력 이미지를 보고 **문제의 텍스트를 최대한 원문 그대로 추출**해야 해.

    입력 이미지에는 다음이 포함될 수 있어:
    - 문학 문제의 질문 문장
    - ①~⑤ 보기 선택지
    - 경우에 따라, 질문 문장 **뒤에** 제시되는 '<보기>' 문장

    🔹 반드시 지켜야 할 출력 규칙:

    1. 이미지에 '<보기>'가 존재하면, 질문 문장 **뒤에** <보기> 전체 내용을 정확히 포함해야 해.
    2. <보기>가 없다면, 질문 문장 다음에 바로 선택지를 출력해.
    3. 선택지는 항상 ①~⑤ 모두 빠짐없이 출력할 것.
    4. 출력 형식은 아래와 같아야 해:

    (질문 문장)

    <보기>
    (보기 내용)

    ① ...
    ② ...
    ③ ...
    ④ ...
    ⑤ ...

    ❗주의:
    - <보기>가 있다면 반드시 포함하고, **질문 뒤에 위치**시켜야 해.
    - 줄바꿈, 기호, 문장부호 등은 최대한 원문 그대로 복원해.
    - 절대 설명, 해설, 시스템 메시지 등 **추가 텍스트는 포함하지 마**.
    - 오직 이미지 속 문제 텍스트만 출력해.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt.strip()},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url
                        }
                    }
                ]
            }
        ],
        max_tokens=1500,
        temperature=0.1
    )

    return response.choices[0].message.content.strip()

# **<이미지에서 추출한 텍스트로 테스트>**
- 2025학년도(2024년 시행) 6월, 9월 수능 테스트

In [ ]:
import os
import re

# 🔹 정답 키
answer_key = {
    "2024-06-국어": {
        18: "4", 19: "3", 20: "3", 21: "4", 22: "1", 23: "2",
        24: "5", 25: "2", 26: "5", 27: "2", 28: "3", 29: "4",
        30: "3", 31: "5", 32: "4", 33: "3", 34: "3"
    },
    "2024-09-국어": {
        18: "1", 19: "3", 20: "5", 21: "5", 22: "3", 23: "1",
        24: "4", 25: "4", 26: "5", 27: "3", 28: "4", 29: "3",
        30: "5", 31: "2", 32: "1", 33: "3", 34: "3"
    },
    "2024-수능-국어": {
        18: "2", 19: "4", 20: "1", 21: "4", 22: "4", 23: "5",
        24: "2", 25: "2", 26: "1", 27: "1", 28: "4", 29: "3",
        30: "5", 31: "4", 32: "3", 33: "5", 34: "2"
    }
}

# 🔹 지문-문항 범위 매핑
passage_problem_mapping = {
    "2024-06-국어": {"p5": (18, 21), "p6": (22, 26), "p7": (27, 30), "p8": (31, 34)},
    "2024-09-국어": {"p5": (18, 21), "p6": (22, 27), "p7": (28, 31), "p8": (32, 34)},
    "2024-수능-국어": {"p5": (18, 21), "p6": (22, 27), "p7": (28, 31), "p8": (32, 34)},
}

# 🔹 OCR 및 평가 실행
def extract_and_evaluate_passage_and_questions(base_path, exam_name, page):
    if exam_name not in passage_problem_mapping or page not in passage_problem_mapping[exam_name]:
        print(f"❌ 매핑 정보 없음: {exam_name} {page}")
        return

    start, end = passage_problem_mapping[exam_name][page]
    passage_path = os.path.join(base_path, f"{exam_name}_{page}.png")
    if not os.path.exists(passage_path):
        print(f"\n⚠️ 지문 이미지 없음: {exam_name}_{page}.png")
        return

    print(f"\n📘 지문: {exam_name} {start}~{end}")
    print("=" * 60)

    passage_text = run_split_pipeline(passage_path)

    for num in range(start, end + 1):
        question_path = os.path.join(base_path, f"{exam_name}_{num}.png")
        if not os.path.exists(question_path):
            print(f"⚠️ 문제 이미지 없음: {exam_name}_{num}.png")
            continue

        print(f"\n--- ❓ 문제 {num} ---")
        question_text = extract_question(question_path)

        # 모델 응답 및 정답 비교
        correct = answer_key.get(exam_name, {}).get(num, "❓")
        try:
            print(f"📌 예측 결과: {tutor_response(question=question_text, passage=passage_text)}")
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
        print(f"✅ 정답: {correct}")

# 🔹 전체 회차 실행
def run_tutor_inference(base_path, year, sessions=None):
    sessions = sessions or ["06", "09", "수능"]

    for 회차 in sessions:
        exam_name = f"{year}-{회차}-국어"
        for page in ["p5", "p6", "p7", "p8"]:
            extract_and_evaluate_passage_and_questions(base_path, exam_name, page)

In [ ]:
base_path = "/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/문학/data/output_images"

# 전체 연도 평가
# run_tutor_inference(base_path, "2022")

# 2024년 6, 9, 수능만 평가
run_tutor_inference(base_path, "2024", sessions=["06"])


📘 지문: 2024-06-국어 18~21
📂 이미지 로딩 중...
🔀 이미지 4등분 중...
🔍 [1/4] 분할 이미지 OCR 중...
🔍 [2/4] 분할 이미지 OCR 중...
🔍 [3/4] 분할 이미지 OCR 중...
🔍 [4/4] 분할 이미지 OCR 중...
🔧 GPT 정교화 단계 진행 중...
🧠 특수기호 위치 검토 및 보정 중...
🗂️ GPT 지문 범위 표시([A], [B] 등) 삽입 중...

--- ❓ 문제 18 ---
📘 [문제에 대한 정답 및 해설]


/tmp/ipython-input-7-3533957196.py:50: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(f"{context}\n\n{question}")


[정답]
- ③ ㉢은 장 한림 부부가 간신의 모해로 유배 간 공간이다.

[해설]
- 문제에서 요구한 것은 ㉠~㉤의 각 공간에 대한 설명입니다. 지문을 통해 각 공간의 역할과 사건을 파악해야 합니다.

1. ① ㉠은 이대봉이 이름의 영혼을 만나 갑옷과 칼을 얻은 공간이다.
   - ㉠은 장 한림의 딸 애향이 제문을 읽는 장면으로, 이대봉이 영혼을 만나 갑옷과 칼을 얻은 공간이 아닙니다. 따라서 틀렸습니다.

2. ② ㉡은 흉노가 침범한 곳이자 이대봉이 흉노를 처단한 공간이다.
   - ㉡은 용궁을 떠나 황성에 올라와 머물 곳을 정한 후의 장면으로, 흉노가 침범한 곳과는 관련이 없습니다. 따라서 틀렸습니다.

3. ③ ㉢은 장 한림 부부가 간신의 모해로 유배 간 공간이다.
   - 지문에서 "부친은 대인의 억울함과 소첩의 안길이 그릇됨을 원통히 여겨 걱정과 분노가 병이 되어 중도에 세상을 버리시니"라는 부분에서 장 한림 부부가 유배 간 사실이 언급됩니다. 따라서 맞습니다.

4. ④ ㉣은 이대봉이 중원을 향하기 전에 머물던 공간이다.
   - ㉣은 금릉으로 행차하는 장면으로, 중원을 향하기 전에 머물던 공간이 아닙니다. 따라서 틀렸습니다.

5. ⑤ ㉤은 동돌수가 이대봉을 피해 달아난 공간이다.
   - ㉤은 지문에 직접적으로 언급되지 않았으며, 동돌수가 달아난 공간이라는 설명은 지문과 일치하지 않습니다. 따라서 틀렸습니다.
📌 예측 결과: None
✅ 정답: 4

--- ❓ 문제 19 ---
📘 [문제에 대한 정답 및 해설]
[정답]
- ③ 부친이 ‘세상을 버린’ 까닭은 혼약이 어그러진 것과 이 시랑의 죽음에 대한 분노 때문이라고 여겼다.

[해설]
- ③번 선택지는 지문에서 부친이 세상을 떠난 이유를 잘못 해석하고 있습니다. 지문에 따르면, "부친은 대인의 억울함과 소첩의 안길이 그릇됨을 원통히 여겨 걱정과 분노가 병이 되어 중도에 세상을 버리시니"라는 부분에서 부친이 세상을 떠난 이유는 이 시랑의 억울함과 혼약이 어그러진 것에 대한 원통함 때문이지, 이 시

In [ ]:
run_tutor_inference(base_path, "2024", sessions=["09"])


📘 지문: 2024-09-국어 18~21
📂 이미지 로딩 중...
🔀 이미지 4등분 중...
🔍 [1/4] 분할 이미지 OCR 중...
🔍 [2/4] 분할 이미지 OCR 중...
🔍 [3/4] 분할 이미지 OCR 중...
🔍 [4/4] 분할 이미지 OCR 중...
🔧 GPT 정교화 단계 진행 중...
🧠 특수기호 위치 검토 및 보정 중...
🗂️ GPT 지문 범위 표시([A], [B] 등) 삽입 중...

--- ❓ 문제 18 ---
📘 [문제에 대한 정답 및 해설]
[정답]
- ① 이도련은 춘향이 자신에게 호감을 느꼈다는 사실을 알지 못했다.

[해설]
- ① 선택지는 이도련이 춘향의 호감을 인지하지 못했다는 내용을 담고 있습니다. 지문에서 이도련은 춘향의 아름다움에 감탄하며 그녀를 부르려 하지만, 춘향이 이도련에게 호감을 느꼈다는 직접적인 언급은 없습니다. 따라서 이도련이 춘향의 호감을 알지 못했다는 해석이 적절합니다.

- ② 선택지는 춘향이 그네를 타기 위해 나들이에 나섰지만 기대했던 바를 달성하지 못했다고 설명합니다. 그러나 지문에서는 춘향이 그네를 타고 즐기는 모습이 묘사되어 있으며, 기대했던 바를 달성하지 못했다는 내용은 없습니다.

- ③ 선택지는 이도련이 김한의 말을 믿었다고 설명하지만, 지문에서는 이도련이 김한의 말을 믿었다는 명확한 근거가 없습니다. 오히려 김한이 춘향을 불러오라는 이도련의 명령을 따르지 않고 춘향이 오지 않을 것이라고 말하는 장면이 있습니다.

- ④ 선택지는 이도련이 월매가 춘향의 어머니라는 사실을 알고 있었다고 설명하지만, 지문에서는 이도련이 김한의 설명을 듣고 월매의 딸이 춘향이라는 사실을 알게 되는 장면이 있습니다. 따라서 이도련이 이를 모르는 척했다는 설명은 부적절합니다.

- ⑤ 선택지는 옆집 여자 아이가 이도련을 만나기 위해 춘향과 함께 왔다고 설명하지만, 지문에서는 옆집 여자 아이가 풍경을 즐기기 위해 나섰다고 설명하고 있습니다. 이도련을 만나기 위해 왔다는 내용은 지문에 없습니다.
📌 예측 결과: None

In [ ]:
run_tutor_inference(base_path, "2024", sessions=["수능"])


📘 지문: 2024-수능-국어 18~21
📂 이미지 로딩 중...
🔀 이미지 4등분 중...
🔍 [1/4] 분할 이미지 OCR 중...
🔍 [2/4] 분할 이미지 OCR 중...
🔍 [3/4] 분할 이미지 OCR 중...
🔍 [4/4] 분할 이미지 OCR 중...
🔧 GPT 정교화 단계 진행 중...
🧠 특수기호 위치 검토 및 보정 중...
🗂️ GPT 지문 범위 표시([A], [B] 등) 삽입 중...

--- ❓ 문제 18 ---
📘 [문제에 대한 정답 및 해설]
[정답]
- ⑤

[해설]
- ⑤번 선택지는 복록의 상황에 대해 언급하고 있습니다. 그러나 지문에서는 복록이 자신이 지은 '죄'에 대해 심리적 중압감을 느꼈다는 구체적인 내용이 드러나지 않습니다. 따라서 이 선택지는 지문과 일치하지 않는 내용입니다.

- ①번 선택지는 ㉠에서 호첩에게 묻는 '연고'의 내용이 왕비가 말한 '사연'과 관련이 있다는 점을 지적하고 있습니다. 이는 지문에서 승상이 호첩에게 집안의 변고를 물었고, 왕비가 정렬부인의 모략에 대해 말한 것과 관련이 있습니다.

- ②번 선택지는 승상이 황상에게 올린 '상소'의 내용이 '이미 아는 바'와 같다는 점을 언급하고 있습니다. 지문에서 승상은 왕비에게 "이 일은 소자가 이미 아는 바이오니 염려 마옵소서"라고 말하며, 상소를 통해 사건의 전말을 아뢰고 있습니다.

- ③번 선택지는 ㉡에서 승상이 '사연'의 진상을 밝히는 데 왕비가 '그놈'의 행위를 알게 된 경위가 중요하다고 생각했다는 점을 지적하고 있습니다. 이는 지문에서 승상이 왕비에게 "처음에 그놈이 충렬부인 방에 간 줄 어찌 알으셨나이까?"라고 물은 부분과 일치합니다.

- ④번 선택지는 ㉡에 대한 왕비의 대답에서 왕비에게 '그놈'의 행위에 대해 제보한 사람이 있었다는 점을 언급하고 있습니다. 이는 왕비가 "사촌 오라비가 이르기로 알았노라"고 답한 부분과 일치합니다.
📌 예측 결과: None
✅ 정답: 2

--- ❓ 문제 19 ---
📘 [문제에 대한 정답 및 해설]
[정답]


# **<테스트 결과>**

❌ 오답 문항 수

- 2025년 6월: 18, 20, 21, 26, 27, 29, 32 → 7문항

- 2025년 9월: 24, 27 → 2문항

- 2025년 수능: 18, 19, 20, 21, 26, 32 → 6문항

- 총 오답: 15문항

🧾 요약
- 총 문항 수: 51

- 정답 수: 36

- 오답 수: 15

### **정답률: 약 70.59%**
- 깔끔한 텍스트에서 실험했을때 틀렸던 문항들이 이번에도 틀리는 모습을 확인
- 지문 이미지의 텍스트를 잘못 추출했을때 그 지문의 문항들이 틀리는 것 때문에 정답률이 낮아짐